# A Toy Trading System [Test]

I wanted to work on the node class, but that's hard without having an application/demo that is meaty enough to
work with.

So, here's a toy front-office position management, pricing, and risk system:

In [55]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown
from mand.core import num, find
from mand.lib.dbsetup import setUpDb
import datetime

db = ObjectDb()
setUpDb(db)

## Books and Portfolios [BA]

These are just workflow entities that support a net present value function. As long as items in books are valuable
(in the sense of implementing NPV) then everything just works.

In [56]:
class TradingBook(Workbook):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            ret += i.NPV() * q
        return ret

class TradingPortfolio(Portfolio):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            ret += i.NPV() * q
        return ret
    
_tr.add(TradingBook)
_tr.add(TradingPortfolio)

## Building a tree of bank-side accounts [User]

Here, we build a tree of 100 books, grouped under 10 sub-portfolios, rooted in one top-level portfolio.

In a real bank, we might have hundreds of thousands of books, grouped 10 levels deep. in multiple different trees. 
Plus millions of customer books grouped in various portfolio trees.

In [57]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['FX', 'Rates', 'Credit', 'Eq-Prop', 'Delta1', 'Eq-Inst', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

<Entity:/Global/TradingPortfolio/TopOfTheHouse>
# books: 100
# children: 10


## Checking execution counts [Test]

We have 11 portfolios, 100 books. We should get node calculation counts that reflect this...

In [58]:
with ProfileMonitor(mode='sum'):
    db2 = db.copy()
    p = db2._get(pAll.meta.path())
    print len(p.books())

100



### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:books|11|550,748|130|50,068|GetValue
|Portfolio:books|11|550,736|772|50,066|GetValue/Calc
|Portfolio:children|11|549,846|131|49,986|GetValue
|Portfolio:children|11|549,714|46,275|49,974|GetValue/Calc
|PortfolioUpdateEvent:children|11|435,091|1,276|39,553|GetValue
|TradingBook|100|391,164|391,164|3,911|Db.Get
|PortfolioUpdateEvent|11|55,212|55,212|5,019|Db.Get
|TradingPortfolio|10|42,650|42,650|4,265|Db.Get
|Clock:cutoffs|22|8,643|78|392|GetValue
|Clock:cutoffs|1|8,564|31|8,564|GetValue/Calc
|Clock:parent|1|8,529|8|8,529|GetValue
|Clock:parent|1|8,521|3,939|8,521|GetValue/Calc
|CosmicAll|1|4,706|4,706|4,706|Db.Get
|Entity:clock|2|4,495|20|2,247|GetValue
|Entity:clock|1|4,474|25|4,474|GetValue/Calc
|RootClock|1|4,448|4,448|4,448|Db.Get
|Portfolio:clock|22|4,416|200|200|GetValue
|Portfolio:clock|11|4,215|224|383|GetValue/Calc
|Clock|1|3,991|3,991|3,991|Db.Get
|RootClock:cutoffs|3|89|15|29|GetValue
|Event:amends|11|75|75|6|GetValue
|RootClock:cutoffs|1|73|39|73|GetValue/Calc
|RootClock:cosmicAll|1|19|9|19|GetValue
|CosmicAll:dbState|1|15|9|15|GetValue
|RootClock:cosmicAll|1|10|10|10|GetValue/Calc
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

## A few books [Test]

We also create two customers' books.

Pretend *p1* is one of our trading desks, *p2* is another. 

In [59]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[3]
p2 = pAll.children()[5]

b1 = p1.children()[4]
b2 = p2.children()[7]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop4
Eq-Inst7


## Tracking Market Data [DBA]

For now, just save incoming market observations as reference data. 

In reality, we'd have billions of observations per day on millions of data sources.

In [60]:
class MarketDataSource(ExternalRefData):
    @dataField
    def last(self):
        return None
    
_tr.add(MarketDataSource)

## Making some data sources [Test]

Two pretend data sources: IBM and Google last trade prices.

In [61]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

print s1_ibm.last()
print s1_goog.last()

175.61
852.12


## Market Interface [DBA]

How user code accesses market data. 

In reality, market interfaces would be choosing amongst raw data sources,
providing bootstrapped curves, vol surfaces, etc.

In [62]:
class MarketInterface(Entity):
    
    @node
    def sourceName(self):
        return 'source1'
    
    @node
    def source(self):
        return self.getObj(_tr.MarketDataSource, '%s.%s' % (self.sourceName(), self.meta.name()))
    
    @node(tweakable=True)
    def spot(self):
        return self.source().last()
                           
    
_tr.add(MarketInterface)


## Making some market interfaces [Test]

In [63]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

print ibm.spot(), type(ibm.spot())
print goog.spot(), type(goog.spot())

175.61 <class 'decimal.Decimal'>
852.12 <class 'decimal.Decimal'>


## Instruments [BA]

Things we can own/have contracted. Instruments have an NPV.

We implement two classes of instrument:
* ForwardCashflow: money that will arrive at some future time
* Equity: a share of common stock in a company
    
Both implementations are toys.

In [64]:
class Instrument(WorkItem):
    """A thing that can be owned, an asset, or legal obligation"""
    
class ForwardCashflow(Instrument):
    
    @node(stored=True)
    def currency(self):
        return 'USD'
    
    @node(stored=True)
    def settlementDatetime(self):
        d = datetime.datetime.utcnow() + datetime.timedelta(2)
        return d
    
    @node
    def NPV(self):
        # XXX - would really get the currency discount curve here, and discount according to 
        # current time/settlement time
        # XXX - and do a conversion to our native currency
        return 1
    
    @node
    def name(self):
        return 'Cash %s/%s' % (self.currency(), self.settlementDatetime())
    
class Equity(Instrument):
    
    @node(stored=True)
    def assetName(self):
        return 'IBM.Eq.1'
    
    @node
    def NPV(self):
        return self.refdata().spot()
    
    @node
    def refdata(self):
        return self.getObj(_tr.MarketInterface, self.assetName().split('.')[0])

    @node
    def name(self):
        return 'Stock: %s' % self.assetName()
    
_tr.add(ForwardCashflow)
_tr.add(Equity)

## TradeOpenEvent [BA]

An event/observation that represents buying or selling something.

Someone gets *quantity* of *item*, and pays *unitPrice* of *premium* for each *item*.

Note that *premium* is typically a *ForwardCashflow*, so the money settles T+2. From a risk/PnL point
of view, we get the asset immediately. This is a gross over-simplification.

In [65]:
class TradeOpenEvent(WorkItemOpenEvent):
    @node(stored=True)
    def action(self):
        return 'Buy'
    @node(stored=True)
    def quantity(self):
        return 1.
    @node(stored=True)
    def premium(self):
        return None
    @node(stored=True)
    def unitPrice(self):
        return 0.
    
    def _items(self):
        bs = 1 if self.action() == 'Buy' else -1
        pq = -bs * self.unitPrice() * self.quantity()
        return [ [ self.ticket(), self.item(),    bs*self.quantity(), self.book1(), self.book2() ],
                 [ self.ticket(), self.premium(), pq,                 self.book1(), self.book2() ]
               ] 

_tr.add(TradeOpenEvent)

## Let's book some trades [Test]

In [66]:
with db:
    cf1 = ForwardCashflow()
    ins1 = Equity()
    ins2 = Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

# Reporting [User]

Reporting is the whole point of the Abstract Nonsense Db.

We ignore all the standard report infrastructure goop (aggregation, GUIs, pivot-tables, drill-down, etc) and
just focus on extracting the underlying data.

For the following examples, we mostly care about NPV (the present values of what we own,) 
PnL (how much money we made or lost,) and risk (what happens to our NPV if underlying conditions change.)

Note that even in the trivial case of our three trades, one amendment, and several market data updates, trying to write
code to attribute profit, or even trying to figure it by hand, would require assumptions and be error-prone. 

In general, the less a report knows about the underlying data, the better it is.

## Report: PnL [User]

How much money did we (i.e. our portfolio) gain or lose between two times?

For example, if time1 was yesterday's close, and time2 is today's close, this report tells us how much we made or lost
today. Will we be drinking champagne or straight vodka this evening?

Note: it's worth understanding how the cash entries are working in this example. The reason our open and closed out
positions are pricing rationally is because we are modelling cash approximately correctly, not because the instruments
are carrying around a notion of price-paid!

In [67]:
def reportPnL(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    with Context({clock.cutoffs: ts2}):
        npv2 = valuable.NPV()
    displayMarkdown('#### PnL for %s: %s' % (valuable.meta.name(), npv2-npv1))

### Example: top-of-the-house intraday PnL

Hmm, one of our desks bought 100 shares of IBM, and the last trade price is already down 1 cent. The bank is down 
100 times $.01 or one dollar:

In [68]:
# 100 shares bought at $175.65
# current price is $175.64:
reportPnL(pAll, ts1, ts3)

#### PnL for TopOfTheHouse: -1.00

### Example: top-of-the-house daily PnL

By end of day, the bank as a whole has closed out the IBM trade for a profit, and bought some GOOG. Closing refdata
prices have been entered. We should have a total profit for the day of $23:

In [69]:
# 100 * ($175.85 - $175.65) =  $20
# 300 * ($852.13 - $852.12) =  $ 3
#                              ----
#                              $23
reportPnL(pAll, ts1, eod)

#### PnL for TopOfTheHouse: 23.00

### Example: trading desk daily PnL

Desk *p1* bought IBM at $175.65

It still owns IBM, which closed at $175.68

So, it should be up $3:

In [70]:
# 100 * ($175.68 - $175.65) =   $3
reportPnL(p1, ts1, eod)

#### PnL for Eq-Prop: 3.00

## Report: Market Risk [DBA]

So, how much does our value change if market data values change?

Note: review the greeks workbook if this is unclear.

In [71]:
def reportMarketRisk(valuable):
    def fn(node):
        obj = node.key.object()
        m = node.key.shortName()
        if isinstance(obj, MarketInterface) and m == 'spot':
            return True
    nodes = find(valuable.NPV, fn)
    r = {}
    npv = valuable.NPV()
    for n in nodes:
        v = n.value * num(1.01)
        with Context({n: v}):
            obj = n.key.object()
            npv2 = valuable.NPV()
            r[obj.meta.name()] = npv2-npv
    displayMarkdown('### %s: NPV change under a 1%% move' % valuable.meta.name())
    displayDict(r)

In [72]:
reportMarketRisk(pAll)
reportMarketRisk(p1)
reportMarketRisk(p2)

### TopOfTheHouse: NPV change under a 1% move

|key|value|
|-|-|
|GOOG|2568.3900

### Eq-Prop: NPV change under a 1% move

|key|value|
|-|-|
|IBM|177.6800

### Eq-Inst: NPV change under a 1% move

|key|value|
|-|-|
|GOOG|5162.463900
|IBM|-357.136800

## Report: Prior Day Amends [User]

So, someone reported a number on day T. How would that number change if we reran the report based on corrected data?

There are a lot of ways to handle prior day amends:
* Have them entered, but have no systemic way to track them.
 * This is a time-honoured approach. Usually, someone goes to jail when the auditors eventually notice that some trader
declaring profits each day is just amending the trades the next day and actually losing money.
* Make a policy that there are no prior day amends.
 * Well, now at least you can see the PnL.
 * On the downside, you don't get much explanatory power. It's a bit like switching to cash accounting because your
internal business units are gaming accrual accounting to the point you can't even manage the budget.
 * Oh, and you still get fined/go to jail: if the number is your position size, and you exceeded the legal limit on
day T, you can't just pretend you didn't and book your day T over-limit trade on day T+1. External reality does not
change just because you have a bad operational policy.
* Just do it right.
 * This is the approach we take.

In [73]:
def reportRestates(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    print
    ts1r = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
    with Context({clock.cutoffs: ts1r}):
        npv2 = valuable.NPV()
    if npv2-npv1:
        displayMarkdown('#### AUDIT-ITEM. %s: NPV change due to prior day amends is %s' % 
                        (valuable.meta.name(), npv2-npv1))

In [74]:
reportRestates(p1, eod, ts6)

print
print 'Note that we are only looking at amended numbers, not all activity:'
reportPnL(p1, eod, ts6)

#### AUDIT-ITEM. Eq-Prop: NPV change due to prior day amends is -4.00


Note that we are only looking at amended numbers, not all activity:


#### PnL for Eq-Prop: 196.00

# Let's see what is going on when we run a report [Core]

In [75]:
with ProfileMonitor(mode='sum'):
    db3 = db.copy()
    p = db3._get(pAll.meta.path())
    reportPnL(p, ts1, ts6)

#### PnL for TopOfTheHouse: 1219.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Root:4343418776|516|1,418,352|868,742|2,748|Context
|TradingPortfolio:NPV|2|1,418,293|31|709,146|GetValue
|TradingPortfolio:NPV|2|1,418,262|11|709,131|GetValue/Calc
|Portfolio:items|22|1,375,008|204|62,500|GetValue
|Portfolio:items|22|1,374,990|149|62,499|GetValue/Calc
|Workbook:items|200|815,237|1,828|4,076|GetValue
|Workbook:items|200|813,408|1,329|4,067|GetValue/Calc
|Portfolio:children|44|553,541|295|12,580|GetValue
|Portfolio:children|22|553,246|163|25,147|GetValue/Calc
|PortfolioUpdateEvent:children|22|400,996|1,253|18,227|GetValue
|TradingBook|102|368,762|368,762|3,615|Db.Get
|PortfolioUpdateEvent|11|49,910|49,910|4,537|Db.Get
|Equity:NPV|1|43,040|14|43,040|GetValue
|Equity:NPV|1|43,025|7|43,025|GetValue/Calc
|MarketInterface:spot|1|39,103|9|39,103|GetValue
|MarketInterface:spot|1|39,094|9|39,094|GetValue/Calc
|TradingPortfolio|10|37,784|37,784|3,778|Db.Get
|ExternalRefData:state|1|34,745|12|34,745|GetValue
|ExternalRefData:state|1|34,733|10|34,733|GetValue/Calc
|RefData:state|1|34,682|11|34,682|GetValue
|RefData:state|1|34,671|9|34,671|GetValue/Calc
|Clock:cutoffs|446|21,158|1,499|47|GetValue
|RefDataUpdateEvent|4|20,317|20,317|5,079|Db.Get
|Clock:cutoffs|5|19,659|61|3,931|GetValue/Calc
|Clock:parent|5|19,364|44|3,872|GetValue
|Clock:parent|5|19,320|35|3,864|GetValue/Calc
|TradeOpenEvent|4|15,673|15,673|3,918|Db.Get
|_WorkItemEvent:ticket|6|13,338|55|2,223|GetValue
|WorkTicket|3|13,283|13,283|4,427|Db.Get
|Workbook:clock|400|13,017|3,317|32|GetValue
|Clock|3|11,764|11,764|3,921|Db.Get
|Workbook:clock|200|9,699|1,941|48|GetValue/Calc
|_WorkItemEvent:item|3|7,041|44|2,347|GetValue
|Equity|2|6,996|6,996|3,498|Db.Get
|_WorkItemEvent:book2|6|6,845|41|1,140|GetValue
|CosmicAll|1|4,631|4,631|4,631|Db.Get
|Portfolio:clock|44|4,625|355|105|GetValue
|RootClock|1|4,467|4,467|4,467|Db.Get
|Portfolio:clock|22|4,270|195|194|GetValue/Calc
|MarketInterface:source|1|4,261|74|4,261|GetValue
|RefData:clock|2|4,236|15|2,118|GetValue
|RefData:clock|1|4,220|10|4,220|GetValue/Calc
|MarketInterface:source|1|4,187|15|4,187|GetValue/Calc
|MarketDataSource|1|4,090|4,090|4,090|Db.Get
|Equity:refdata|1|3,865|11|3,865|GetValue
|Equity:refdata|1|3,853|8|3,853|GetValue/Calc
|MarketInterface|1|3,799|3,799|3,799|Db.Get
|TradeOpenEvent:premium|3|3,192|30|1,064|GetValue
|ForwardCashflow|1|3,162|3,162|3,162|Db.Get
|Portfolio:books|42|1,371|289|32|GetValue
|Portfolio:books|22|1,139|146|51|GetValue/Calc
|Entity:clock|10|230|87|23|GetValue
|Event:amends|29|187|187|6|GetValue
|Entity:clock|5|143|46|28|GetValue/Calc
|RootClock:cutoffs|15|45|45|3|GetValue
|_WorkItemEvent:book1|6|35|35|5|GetValue
|TradeOpenEvent:quantity|6|29|29|4|GetValue
|ForwardCashflow:NPV|1|22|11|22|GetValue
|RefDataUpdateEvent:data|4|21|21|5|GetValue
|MarketInterface:sourceName|1|19|7|19|GetValue
|TradeOpenEvent:action|3|15|15|5|GetValue
|TradeOpenEvent:unitPrice|3|14|14|4|GetValue
|MarketInterface:sourceName|1|11|7|11|GetValue/Calc
|ForwardCashflow:NPV|1|10|5|10|GetValue/Calc
|Equity:assetName|1|5|5|5|GetValue